## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [34]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss, get_xb
#DSS = get_dss()

In [35]:
XB = get_xb()

In [36]:
PATH = 'data/extra_biblical_datasets/extra_biblical/annotation_df_history/1qs_1qm_1qh_all_verbs.csv' # Modify before executing the notebook

In [37]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [38]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [39]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
#df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['book', 'chapter', 'verse_num'])

# Drop the temporary 'scroll_order' column
#df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [40]:
pd.set_option("display.max_columns", None)

In [41]:
df.head(2)

,index,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_det,cmpl_indiv,cmpl_complex,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4
0,0,206,GLL,1QH,1QH,3,9,HTGWLLTJ,KJ B NDH HTGWLLTJ W M SWD RMH Y>TJ W L> NLWJTJ...,KJ B NDH HTGWLLTJ,,B NDH,B NDH,,0.0,prep,und,subs,simple,,hit,perf,,,B,,,
1,1,354,NPL,1QH,1QH,4,1,HTNPL,MZMWR L MFKJL L HTNPL L PNJ >L M<FJ >L W L HBJ...,L HTNPL L PNJ >L,,L PNJ >L,L PNH >L==,,0.0,prep,und,subs,complex,,hit,infc,,,L,,,


In [42]:
#df[(df.scroll != "MT") & (df.gcons_clause != "None")]

In [43]:
df.columns

Index(['index', 'verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'sign_info', 'stem', 'tense',
       'motion_type', 'comments', 'preposition_1', 'preposition_2',
       'preposition_3', 'preposition_4'],
      dtype='object')

In [17]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [18]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

NameError: name 'dss_sections' is not defined

In [ ]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

In [ ]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)

In [45]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [46]:
df.columns

Index(['index', 'verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'sign_info', 'stem', 'tense',
       'motion_type', 'comments', 'preposition_1', 'preposition_2',
       'preposition_3', 'preposition_4'],
      dtype='object')

In [47]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""
df["Study_Edition"] = ""
new_columns_list = ['cmpl_nt', 'cmpl_anim','reconstructed_verse']

for column_name in new_columns_list:
    df[column_name] = ""

In [48]:
df.columns

Index(['index', 'verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'sign_info', 'stem', 'tense',
       'motion_type', 'comments', 'preposition_1', 'preposition_2',
       'preposition_3', 'preposition_4', 'spatial_arg_type', 'Study_Edition',
       'cmpl_nt', 'cmpl_anim', 'reconstructed_verse'],
      dtype='object')

In [ ]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [49]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN")]
df_test.head(5)
#len(df_test)

,index,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_det,cmpl_indiv,cmpl_complex,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,spatial_arg_type,Study_Edition,cmpl_nt,cmpl_anim,reconstructed_verse


In [50]:
len(df_test)

0

In [51]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index
index_to_drop_2 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "")].index
index_to_drop_3 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "MN") & (df.preposition_4 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)
df = df.drop(index_to_drop_2)
df = df.drop(index_to_drop_3)

# Reset the index
df = df.reset_index(drop=True)

In [52]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,index,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_det,cmpl_indiv,cmpl_complex,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,spatial_arg_type,Study_Edition,cmpl_nt,cmpl_anim,reconstructed_verse


In [53]:
len(df)

308

In [54]:
set(df.scroll)

{'1QH', '1QM', '1QS'}

In [55]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['scroll','chapter', 'verse_num'])

df = df.reset_index(drop=True)

In [56]:
df.head(4)

,index,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_det,cmpl_indiv,cmpl_complex,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,spatial_arg_type,Study_Edition,cmpl_nt,cmpl_anim,reconstructed_verse
0,0,206,GLL,1QH,1QH,3,9,HTGWLLTJ,KJ B NDH HTGWLLTJ W M SWD RMH Y>TJ W L> NLWJTJ...,KJ B NDH HTGWLLTJ,,B NDH,B NDH,,0.0,prep,und,subs,simple,,hit,perf,,,B,,,,,,,,
1,1,354,NPL,1QH,1QH,4,1,HTNPL,MZMWR L MFKJL L HTNPL L PNJ >L M<FJ >L W L HBJ...,L HTNPL L PNJ >L,,L PNJ >L,L PNH >L==,,0.0,prep,und,subs,complex,,hit,infc,,,L,,,,,,,,
2,2,376,HLK,1QH,1QH,4,1,HTHLKW,MZMWR L MFKJL L HTNPL L PNJ >L M<FJ >L W L HBJ...,HTHLKW,,no complement,no complement,,,,,,,,hit,perf,,,,,,,,,,,
3,3,447,GLH,1QH,1QH,4,4,GLJTH,>TH HW>L QDWCJM W B RZJ PL> K HWD< BWR KBWD K ...,>TH GLJTH DRKJ >MT W M<FJ R< XWKMH W >WLT,>TH,no complement,no complement,,,,,,,,qal,perf,,,,,,,,,,,


In [57]:
df.to_csv(PATH, index=False)